In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from resnet import *
#from utils import progress_bar

In [ ]:
#parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
#parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
#parser.add_argument('--resume', '-r', action='store_true',
#                    help='resume from checkpoint')
#args = parser.parse_args()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch


In [ ]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..


100%|██████████| 170498071/170498071 [00:13<00:00, 13000922.15it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
print('==> Building model..')
net = ResNet18()

net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(net.parameters(), lr=0.01, lr_decay=0, 
                        weight_decay=5e-4, initial_accumulator_value=0, eps=1e-08, foreach=None, maximize=False)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

==> Building model..


In [ ]:
pytorch_total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)

In [ ]:
print(pytorch_total_params)

4314954


In [ ]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        #print(batch_idx, len(trainloader), 'Train Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     #% (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    print("Train Loss:",train_loss,"Train acc:",100.*correct/total)

In [ ]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            #print(batch_idx, len(testloader), 'Test Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         #% (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        acc = 100.*correct/total
        if acc > best_acc:
          print('Saving..')
          state = {
              'net': net.state_dict(),
              'acc': acc,
              'epoch': epoch,
          }
          if not os.path.isdir('checkpoint'):
              os.mkdir('checkpoint')
          torch.save(state, './checkpoint/ckpt.pth')
          best_acc = acc
        print("Validation loss",test_loss,"Validation Acc:",acc)

In [ ]:
epoch = 0
while best_acc<90:
    epoch+=1
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 1
Train Loss: 590.2679862380028 Train acc: 44.65
Saving..
Validation loss 161.03817760944366 Validation Acc: 44.38

Epoch: 2
Train Loss: 399.44855386018753 Train acc: 63.288
Saving..
Validation loss 173.45495212078094 Validation Acc: 48.64

Epoch: 3
Train Loss: 331.7838656902313 Train acc: 69.994
Saving..
Validation loss 98.48531764745712 Validation Acc: 66.3

Epoch: 4
Train Loss: 286.65822064876556 Train acc: 74.254
Saving..
Validation loss 79.01605874300003 Validation Acc: 72.41

Epoch: 5
Train Loss: 251.65001973509789 Train acc: 77.42
Validation loss 82.66349440813065 Validation Acc: 71.59

Epoch: 6
Train Loss: 225.50990119576454 Train acc: 79.872
Validation loss 82.62339907884598 Validation Acc: 71.64

Epoch: 7
Train Loss: 207.8714388012886 Train acc: 81.584
Saving..
Validation loss 67.90978035330772 Validation Acc: 77.66

Epoch: 8
Train Loss: 193.27589383721352 Train acc: 82.75
Validation loss 75.0188530087471 Validation Acc: 76.38

Epoch: 9
Train Loss: 180.20036193728447 